We will be using the transformer to summarize the combined_text (pre_text and post_text) and take that summarized data as a context for our QnA Transformer.  

In [1]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration

model_name = "human-centered-summarization/financial-summarization-pegasus"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

c:\Users\sehaj\OneDrive\Desktop\Capstone\FinChatbot\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: 
PegasusTokenizer requires the SentencePiece library but it was not found in your environment. Checkout the instructions on the
installation page of its repo: https://github.com/google/sentencepiece#installation and follow the ones
that match your environment. Please note that you may need to restart your runtime after installation.


In [2]:
def summarize_text(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=1024)
    summary_ids = model.generate(inputs.input_ids, num_beams=5, max_length=150, min_length=50, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [3]:
import pandas as pd

In [4]:
df = pd.read_json("../Data/FinQ&A/train_t.json")

In [5]:
df2 = df.head(7).copy()

In [6]:
df2.shape

(7, 10)

In [16]:
def flatten_list(nested_list):
    # Join characters within each sublist without spaces, then join sublists with a space
    return " ".join(["".join(sublist) for sublist in nested_list])

# Apply the function to flatten pre_text and post_text
df2["combined_text"] = df2["pre_text"].apply(flatten_list) + " " + df2["post_text"].apply(flatten_list)

In [17]:
df2

,pre_text,post_text,filename,table_ori,table,qa,id,annotation,qa_0,qa_1,combined_text
0,"[26 | 2009 annual report in fiscal 2008 , reve...","[year ended june 30 , cash provided by operati...",JKHY/2009/page_28.pdf,"[[, Year ended June 30, 2009], [2008, 2007], [...","[[2008, year ended june 30 2009 2008, year end...",{'question': 'what was the percentage change i...,Single_JKHY/2009/page_28.pdf-3_0,{'amt_table': '<table class='wikitable'><tr><t...,NaN,NaN,"26 | 2009 annual report in fiscal 2008 , reven..."
1,"[26 | 2009 annual report in fiscal 2008 , reve...","[year ended june 30 , cash provided by operati...",JKHY/2009/page_28.pdf,"[[, Year ended June 30, 2009], [2008, 2007], [...","[[2008, year ended june 30 2009 2008, year end...",{'question': 'what was the percentage change i...,Single_JKHY/2009/page_28.pdf-3_1,{'amt_table': '<table class='wikitable'><tr><t...,NaN,NaN,"26 | 2009 annual report in fiscal 2008 , reven..."
2,"[26 | 2009 annual report in fiscal 2008 , reve...","[year ended june 30 , cash provided by operati...",JKHY/2009/page_28.pdf,"[[, Year ended June 30, 2009], [2008, 2007], [...","[[2008, year ended june 30 2009 2008, year end...",{'question': 'what was the percentage change i...,Single_JKHY/2009/page_28.pdf-3_2,{'amt_table': '<table class='wikitable'><tr><t...,NaN,NaN,"26 | 2009 annual report in fiscal 2008 , reven..."
3,"[26 | 2009 annual report in fiscal 2008 , reve...","[year ended june 30 , cash provided by operati...",JKHY/2009/page_28.pdf,"[[, Year ended June 30, 2009], [2008, 2007], [...","[[2008, year ended june 30 2009 2008, year end...",{'question': 'what was the percentage change i...,Single_JKHY/2009/page_28.pdf-3_3,{'amt_table': '<table class='wikitable'><tr><t...,NaN,NaN,"26 | 2009 annual report in fiscal 2008 , reven..."
4,[substantially all of the goodwill and other i...,[the above unaudited pro forma financial infor...,RSG/2008/page_114.pdf,"[[, Year Ended December 31, 2008 (Unaudited), ...","[[, year ended december 31 2008 ( unaudited ),...",{'question': 'what was the percent of the grow...,Single_RSG/2008/page_114.pdf-2_0,{'amt_table': '<table class='wikitable'><tr><t...,NaN,NaN,substantially all of the goodwill and other in...
5,[substantially all of the goodwill and other i...,[the above unaudited pro forma financial infor...,RSG/2008/page_114.pdf,"[[, Year Ended December 31, 2008 (Unaudited), ...","[[, year ended december 31 2008 ( unaudited ),...",{'question': 'what was the percent of the grow...,Single_RSG/2008/page_114.pdf-2_1,{'amt_table': '<table class='wikitable'><tr><t...,NaN,NaN,substantially all of the goodwill and other in...
6,[substantially all of the goodwill and other i...,[the above unaudited pro forma financial infor...,RSG/2008/page_114.pdf,"[[, Year Ended December 31, 2008 (Unaudited), ...","[[, year ended december 31 2008 ( unaudited ),...",{'question': 'what was the percent of the grow...,Single_RSG/2008/page_114.pdf-2_2,{'amt_table': '<table class='wikitable'><tr><t...,NaN,NaN,substantially all of the goodwill and other in...


In [21]:
df2 = df2.drop(["pre_text", "post_text"], axis = 1)

In [28]:
def summarize_text(text):
    # Reduce the maximum length for both input and output
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=400)  # Reduced from 1024
    summary_ids = model.generate(
        inputs.input_ids,
        num_beams=5,
        max_length=256,  # Reduced from 150
        min_length=30,   # Reduced from 50
        early_stopping=True
    )
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Apply the summarization
try:
    df2["summary"] = df2["combined_text"].apply(summarize_text)
except Exception as e:
    print(f"Error during summarization: {e}")

In [27]:
df2["combined_text"][0]

'26 | 2009 annual report in fiscal 2008 , revenues in the credit union systems and services business segment increased 14% ( 14 % ) from fiscal 2007 . all revenue components within the segment experienced growth during fiscal 2008 . license revenue generated the largest dollar growth in revenue as episys ae , our flagship core processing system aimed at larger credit unions , experienced strong sales throughout the year . support and service revenue , which is the largest component of total revenues for the credit union segment , experienced 34 percent growth in eft support and 10 percent growth in in-house support . gross profit in this business segment increased $ 9344 in fiscal 2008 compared to fiscal 2007 , due primarily to the increase in license revenue , which carries the highest margins . liquidity and capital resources we have historically generated positive cash flow from operations and have generally used funds generated from operations and short-term borrowings on our revol

In [29]:
df2["summary"][0]

'License revenue generated the largest dollar in revenue as episys ae. support and service revenue grew 34 percent in fiscal 2008 compared to fiscal 2007'

The transformer is not working as expected thus, we will be using some other method to do it.